# Объединённый ноутбук для сбора, анализа и визуализации погодных данных

In [2]:
!pip install wordcloud plotly seaborn

In [1]:
import requests
import pandas as pd
import os
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

In [3]:
# Конфигурация
API_KEY = 'e19341656f7539e9ac6553c950159dc4'
CITIES = {
    'Санкт-Петербург': {'lat': 59.93, 'lon': 30.31},
    'Москва': {'lat': 55.75, 'lon': 37.62},
    'Самара': {'lat': 53.20, 'lon': 50.15}
}

In [5]:
for city, coords in CITIES.items():
    url = (
        f"https://api.openweathermap.org/data/2.5/forecast"
        f"?lat={coords['lat']}&lon={coords['lon']}&appid={API_KEY}&units=metric&lang=ru"
    )

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Ошибка при получении данных для {city}: {response.status_code}")
        continue

    data = response.json()

    df = pd.DataFrame([{
        "datetime": pd.to_datetime(entry["dt"], unit="s"),
        "temp_c": entry["main"]["temp"],
        "feels_like_c": entry["main"]["feels_like"],
        "humidity": entry["main"]["humidity"],
        "pressure": entry["main"]["pressure"],
        "wind_speed_kmh": round(entry["wind"]["speed"] * 3.6, 1),
        "weather": entry["weather"][0]["description"],
        "rain_mm": entry.get("rain", {}).get("3h", 0)
    } for entry in data.get("list", [])])

    filename = f"weather_{city}_realtime.csv"
    df.to_csv(filename, index=False)
    print(f"Файл для {city} сохранён!")

    files.download(filename)

Файл для Санкт-Петербург сохранён!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Файл для Москва сохранён!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Файл для Самара сохранён!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# Вычисление статистики
def compute_statistics(df):
    if df.empty:
        raise ValueError("DataFrame пуст. Проверьте загруженные данные.")
    stats = df.groupby('city').agg({
        'temp_c': ['mean', 'min', 'max'],
        'rain_mm': 'sum'
    })
    stats.columns = ['avg_temp', 'min_temp', 'max_temp', 'total_rain']
    return stats.reset_index()

stats = compute_statistics(df_all)
print(stats)

              city  avg_temp  min_temp  max_temp  total_rain
0           Москва  13.13575      8.54     19.28       17.27
1           Самара  15.52375      9.88     21.54        7.61
2  Санкт-Петербург  10.35300      5.27     15.82        3.38


In [8]:
# Генерация визуализаций
os.makedirs('dashboard/figures', exist_ok=True)

def plot_temperature_trend(df):
    plt.figure(figsize=(14, 7))
    sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
    ax = sns.lineplot(
        data=df,
        x='datetime',
        y='temp_c',
        hue='city',
        style='city',
        markers=True
    )
    plt.title('Динамика температуры по городам', fontsize=16, pad=20)
    plt.xlabel('Дата и время', fontsize=12)
    plt.ylabel('Температура (°C)', fontsize=12)
    plt.xticks(rotation=45)
    plt.legend(title='Город', title_fontsize=12, fontsize=11, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig('dashboard/figures/temperature_trend.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_weather_wordcloud(df):
    from collections import Counter
    weather_text = ' '.join(df['weather'].dropna())
    word_counts = Counter(weather_text.split())
    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        colormap='viridis',
        max_words=50
    ).generate_from_frequencies(word_counts)
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title('Частота погодных условий', fontsize=16, pad=20)
    plt.axis('off')
    plt.savefig('dashboard/figures/weather_wordcloud.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_interactive_temp_map(df):
    avg_temp = df.groupby('city', as_index=False)['temp_c'].mean()
    city_coords = {
        'Москва': [55.7558, 37.6173],
        'Санкт-Петербург': [59.9343, 30.3351],
        'Самара': [53.1951, 50.1009]
    }
    avg_temp['lat'] = avg_temp['city'].map(lambda x: city_coords[x][0])
    avg_temp['lon'] = avg_temp['city'].map(lambda x: city_coords[x][1])
    fig = px.scatter_mapbox(
        avg_temp,
        lat='lat',
        lon='lon',
        size='temp_c',
        color='temp_c',
        hover_name='city',
        hover_data={'temp_c': ':.1f'},
        size_max=30,
        zoom=3,
        color_continuous_scale=px.colors.sequential.Tealgrn,
        title='Средняя температура по городам'
    )
    fig.update_layout(
        mapbox_style="open-street-map",
        margin={'r':0,'t':40,'l':0,'b':0},
        title_x=0.5,
        title_font_size=20
    )
    fig.write_html('dashboard/figures/interactive_temp_map.html')

def generate_visualizations(df):
    plot_temperature_trend(df)
    plot_weather_wordcloud(df)
    plot_interactive_temp_map(df)
    print("Визуализации успешно сохранены в папку dashboard/figures/")

generate_visualizations(df_all)

Визуализации успешно сохранены в папку dashboard/figures/
